In [1]:
import numpy as np
import lime
import torch
import torch.nn.functional as F
from lime.lime_text import LimeTextExplainer

In [2]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification



In [3]:
from transformers import AutoModelForSequenceClassification
from transformers import AutoTokenizer
import numpy as np
import pandas as pd
from typing import List

# this is the name of the model we want to evaluate on 
# huggingface.com/models or alternatively you could train your own
MODEL="nlptown/bert-base-multilingual-uncased-sentiment"

tokenizer = AutoTokenizer.from_pretrained(MODEL)
model = AutoModelForSequenceClassification.from_pretrained(MODEL)

In [4]:
def model_adapter(texts: List[str]):
    
    all_scores = []

    for i in range(0, len(texts), 64):

        batch = texts[i:i+64]
        
        # use bert encoder to tokenize text 
        encoded_input = tokenizer(batch, 
          return_tensors='pt', 
          padding=True, 
          truncation=True, 
          max_length=model.config.max_position_embeddings-2)

        # run the model
        output = model(**encoded_input)
        # by default this model gives raw logits rather 
        # than a nice smooth softmax so we apply it ourselves here
        scores = output[0].softmax(1).detach().numpy()

        all_scores.extend(scores)

    return np.array(all_scores)

In [5]:
target_names=list(model.config.id2label.values())

In [6]:
target_names

['1 star', '2 stars', '3 stars', '4 stars', '5 stars']

In [7]:
explainer = LimeTextExplainer(class_names=target_names)

In [8]:
explainer

In [11]:
# str_to_predict = "surprising increase in revenue in spite of decrease in market share"
# exp = explainer.explain_instance(str_to_predict, model_adapter, num_features=5, num_samples=2000)

In [19]:
from eli5.lime import TextExplainer

te = TextExplainer(n_samples=5000, random_state=42)
te.fit("""The restaurant was amazing, the quality of their 
food was exceptional. The waiters were so polite.""", model_adapter)
te.explain_prediction(target_names=list(model.config.id2label.values()))

Explanation(estimator="SGDClassifier(alpha=0.001, loss='log', penalty='elasticnet',\n              random_state=RandomState(MT19937) at 0x23444ACF740)", description=None, error=None, method='linear model', is_regression=False, targets=[TargetExplanation(target='1 star', feature_weights=FeatureWeights(pos=[FeatureWeight(feature='were so', weight=0.4412142792788092, std=None, value=1.0), FeatureWeight(feature='food was', weight=0.26752107341372183, std=None, value=1.0), FeatureWeight(feature='restaurant was', weight=0.266566087992046, std=None, value=1.0), FeatureWeight(feature='waiters were', weight=0.20671345225968546, std=None, value=1.0), FeatureWeight(feature='their food', weight=0.19225542650789543, std=None, value=1.0), FeatureWeight(feature='of their', weight=0.17089439768401474, std=None, value=1.0), FeatureWeight(feature='so polite', weight=0.16971856137145255, std=None, value=1.0), FeatureWeight(feature='the restaurant', weight=0.09395728540023118, std=None, value=1.0), FeatureWeight(feature='the quality', weight=0.06235816727168654, std=None, value=1.0), FeatureWeight(feature='was amazing', weight=0.04621754955053717, std=None, value=1.0), FeatureWeight(feature='quality of', weight=0.036982483291333315, std=None, value=1.0)], neg=[FeatureWeight(feature='exceptional', weight=-1.3083460073861428, std=None, value=1.0), FeatureWeight(feature='amazing', weight=-1.2489714603173006, std=None, value=1.0), FeatureWeight(feature='quality', weight=-0.9791490505974169, std=None, value=1.0), FeatureWeight(feature='restaurant', weight=-0.7171043876100933, std=None, value=1.0), FeatureWeight(feature='food', weight=-0.6679038114428723, std=None, value=1.0), FeatureWeight(feature='their', weight=-0.5886911093430724, std=None, value=1.0), FeatureWeight(feature='waiters', weight=-0.5624537628909229, std=None, value=1.0), FeatureWeight(feature='the', weight=-0.561238646099185, std=None, value=3.0), FeatureWeight(feature='polite', weight=-0.4909223929633988, std=None, value=1.0), FeatureWeight(feature='so', weight=-0.474632552661918, std=None, value=1.0), FeatureWeight(feature='<BIAS>', weight=-0.46817511726941313, std=None, value=1.0), FeatureWeight(feature='were', weight=-0.3105891970589915, std=None, value=1.0), FeatureWeight(feature='was', weight=-0.20789884677179046, std=None, value=2.0), FeatureWeight(feature='of', weight=-0.12298547156907863, std=None, value=1.0), FeatureWeight(feature='was exceptional', weight=-0.07927481966138583, std=None, value=1.0)], pos_remaining=0, neg_remaining=0), proba=0.0009819658649170202, score=-6.833937869621569, weighted_spans=WeightedSpans(docs_weighted_spans=[DocWeightedSpans(document='the restaurant was amazing, the quality of their \nfood was exceptional. the waiters were so polite.', spans=[('the', [(0, 3)], -0.561238646099185), ('restaurant', [(4, 14)], -0.7171043876100933), ('was', [(15, 18)], -0.20789884677179046), ('amazing', [(19, 26)], -1.2489714603173006), ('the', [(28, 31)], -0.561238646099185), ('quality', [(32, 39)], -0.9791490505974169), ('of', [(40, 42)], -0.12298547156907863), ('their', [(43, 48)], -0.5886911093430724), ('food', [(50, 54)], -0.6679038114428723), ('was', [(55, 58)], -0.20789884677179046), ('exceptional', [(59, 70)], -1.3083460073861428), ('the', [(72, 75)], -0.561238646099185), ('waiters', [(76, 83)], -0.5624537628909229), ('were', [(84, 88)], -0.3105891970589915), ('so', [(89, 91)], -0.474632552661918), ('polite', [(92, 98)], -0.4909223929633988), ('the restaurant', [(0, 3), (4, 14)], 0.09395728540023118), ('restaurant was', [(4, 14), (15, 18)], 0.266566087992046), ('was amazing', [(15, 18), (19, 26)], 0.04621754955053717), ('the quality', [(28, 31), (32, 39)], 0.06235816727168654), ('quality of', [(32, 39), (40, 42)], 0.036982483291333315), ('of their', [(40, 42), (43, 48)], 0.17089439768401474), ('their food', [(43, 48), (50, 54)], 0.19225542650789543), ('food was', [(50, 54), (55, 58)], 0.26752107341372183), ('was exceptional', [(55, 58), (59, 70)], -0.079274819

In [8]:
# import numpy as np
# import lime
# import torch
# import torch.nn.functional as F
# from lime.lime_text import LimeTextExplainer

# from transformers import AutoTokenizer, AutoModelForSequenceClassification

# tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")
# model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")
# class_names = ['positive','negative', 'neutral']

# def predictor(texts):
#     outputs = model(**tokenizer(texts, return_tensors="pt", padding=True))
#     probas = F.softmax(outputs.logits).detach().numpy()
#     return probas

# explainer = LimeTextExplainer(class_names=class_names)

# str_to_predict = "surprising increase in revenue in spite of decrease in market share"
# exp = explainer.explain_instance(str_to_predict, predictor, num_features=20, num_samples=2000)
# exp.show_in_notebook(text=str_to_predict)